In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [10]:
# read all the CSV files and combine them into a single dataframe
path = 'C:/Users/pdharantej/OneDrive - ALLEGIS GROUP/Desktop/TEK_Training/Assignments/Data_Analysis/Spark_SQL/Problem2-dataset/'
df_jan = spark.read.option("header",True).csv(path+'Sales_January_01_2021.csv')
df_feb = spark.read.option("header",True).csv(path+"Sales_February_01_2021.csv")
df_mar = spark.read.option("header",True).csv(path+"Sales_March_01_2021.csv")
df_apr = spark.read.option("header",True).csv(path+"Sales_April_01__2021.csv")

df_sales = df_jan.union(df_feb).union(df_mar).union(df_apr)


In [11]:
df_sales.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  141234|              iPhone|               1|       700|01/22/19 21:25|944 Walnut St, Bo...|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|
|  141236|    Wired Headphones|               2|     11.99|01/17/19 13:33|538 Adams St, San...|
|  141237|    27in FHD Monitor|               1|    149.99|01/05/19 20:33|738 10th St, Los ...|
|  141238|    Wired Headphones|               1|     11.99|01/25/19 11:59|387 10th St, Aust...|
|  141239|AAA Batteries (4-...|               1|      2.99|01/29/19 20:22|775 Willow St, Sa...|
|  141240|27in 4K Gaming Mo...|               1|    389.99|01/26/19 12:16|979 Park St, Los ...|
|  141241|USB-C Charging Cable|         

In [26]:
from pyspark.sql.functions import month, to_date

df_sales = df_sales.withColumn("Order Date", to_date("Order Date", "MM/dd/yy HH:mm"))
df_sales = df_sales.withColumn("Month", month("Order Date"))
df_sales = df_sales.withColumn('Month', date_format('Order Date', 'MMM'))
df_sales.show()


+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|Order ID|             Product|Quantity Ordered|Price Each|Order Date|    Purchase Address|Month| Sales|          City|Hour|
+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|  141234|              iPhone|               1|       700|2019-01-22|944 Walnut St, Bo...|  Jan| 700.0|        Boston|   0|
|  141235|Lightning Chargin...|               1|     14.95|2019-01-28|185 Maple St, Por...|  Jan| 14.95|      Portland|   0|
|  141236|    Wired Headphones|               2|     11.99|2019-01-17|538 Adams St, San...|  Jan| 23.98| San Francisco|   0|
|  141237|    27in FHD Monitor|               1|    149.99|2019-01-05|738 10th St, Los ...|  Jan|149.99|   Los Angeles|   0|
|  141238|    Wired Headphones|               1|     11.99|2019-01-25|387 10th St, Aust...|  Jan| 11.99|        Austin|   0|


In [27]:
from pyspark.sql.functions import col

df_sales = df_sales.withColumn("Sales", col("Quantity Ordered") * col("Price Each"))
df_sales.show()

+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|Order ID|             Product|Quantity Ordered|Price Each|Order Date|    Purchase Address|Month| Sales|          City|Hour|
+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|  141234|              iPhone|               1|       700|2019-01-22|944 Walnut St, Bo...|  Jan| 700.0|        Boston|   0|
|  141235|Lightning Chargin...|               1|     14.95|2019-01-28|185 Maple St, Por...|  Jan| 14.95|      Portland|   0|
|  141236|    Wired Headphones|               2|     11.99|2019-01-17|538 Adams St, San...|  Jan| 23.98| San Francisco|   0|
|  141237|    27in FHD Monitor|               1|    149.99|2019-01-05|738 10th St, Los ...|  Jan|149.99|   Los Angeles|   0|
|  141238|    Wired Headphones|               1|     11.99|2019-01-25|387 10th St, Aust...|  Jan| 11.99|        Austin|   0|


In [28]:
from pyspark.sql.functions import split

df_sales = df_sales.withColumn("City", split(col("Purchase Address"), ",")[1])
df_sales.show()

+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|Order ID|             Product|Quantity Ordered|Price Each|Order Date|    Purchase Address|Month| Sales|          City|Hour|
+--------+--------------------+----------------+----------+----------+--------------------+-----+------+--------------+----+
|  141234|              iPhone|               1|       700|2019-01-22|944 Walnut St, Bo...|  Jan| 700.0|        Boston|   0|
|  141235|Lightning Chargin...|               1|     14.95|2019-01-28|185 Maple St, Por...|  Jan| 14.95|      Portland|   0|
|  141236|    Wired Headphones|               2|     11.99|2019-01-17|538 Adams St, San...|  Jan| 23.98| San Francisco|   0|
|  141237|    27in FHD Monitor|               1|    149.99|2019-01-05|738 10th St, Los ...|  Jan|149.99|   Los Angeles|   0|
|  141238|    Wired Headphones|               1|     11.99|2019-01-25|387 10th St, Aust...|  Jan| 11.99|        Austin|   0|


In [29]:
# 1. What was the best month for sales? 


from pyspark.sql.functions import month, sum

sales_by_month = df_sales.groupBy("Month").agg(sum("Sales").alias("Total Sales"))
sales_by_month.orderBy("Total Sales", ascending=False).show()


+-----+------------------+
|Month|       Total Sales|
+-----+------------------+
|  Apr|3390670.2400007015|
|  Mar|2807100.3800003603|
|  Feb| 2202022.419999963|
|  Jan|1813586.4399999138|
|  May|10559.289999999999|
| null|              null|
+-----+------------------+



In [30]:
# 2. How much was earned the month that had highest sales?


from pyspark.sql.functions import month, sum

sales_by_month = df_sales.groupBy("Month").agg(sum("Sales").alias("Total Sales"))
best_month = sales_by_month.orderBy("Total Sales", ascending=False).first()

print(f"The month with the highest sales was {best_month['Month']}, with total sales of {best_month['Total Sales']}.")


The month with the highest sales was Apr, with total sales of 3390670.2400007015.


In [31]:
# 3. What city has the highest sales?



from pyspark.sql.functions import sum

sales_by_city = df_sales.groupBy("City").agg(sum("Sales").alias("Total Sales"))
best_city = sales_by_city.orderBy("Total Sales", ascending=False).first()

print(f"The city with the highest sales was {best_city['City']}, with total sales of {best_city['Total Sales']}.")


The city with the highest sales was  San Francisco, with total sales of 2490980.509999955.


In [32]:
# 4. What time should we display advertisements to maximize likehood of 
# customers buying products?


from pyspark.sql.functions import hour

# Extract the hour from the "Order Date" column
df_sales = df_sales.withColumn("Hour", hour("Order Date"))

# Group the data by hour and count the number of orders for each hour
df_hourly_orders = df_sales.groupBy("Hour").count().orderBy("Hour")

# Show the results
df_hourly_orders.show()


+----+-----+
|Hour|count|
+----+-----+
|null|  258|
|   0|55110|
+----+-----+



In [33]:
# 5. What products are most often sold together?



concat_udf = udf(lambda x: ', '.join(sorted(x)), StringType())
df_sales = df_sales.withColumn('Products', concat_ws(',', 'Product', 'Order ID'))
order_products = df_sales.groupBy('Order ID').agg(collect_set('Products').alias('Product_Sets'))

In [35]:
order_products.show(5,truncate=False)

+--------+---------------------------------+
|Order ID|Product_Sets                     |
+--------+---------------------------------+
|null    |[]                               |
|141234  |[iPhone,141234]                  |
|141235  |[Lightning Charging Cable,141235]|
|141236  |[Wired Headphones,141236]        |
|141237  |[27in FHD Monitor,141237]        |
+--------+---------------------------------+
only showing top 5 rows



In [38]:
product_pairs = order_products.selectExpr('Product_Sets as products')\
                                .groupBy('products').count()\
                                .orderBy('count', ascending=False)

In [39]:
product_pairs.show(5, truncate=False)

+---------------------------------------------------------------------------+-----+
|products                                                                   |count|
+---------------------------------------------------------------------------+-----+
|[iPhone,141336]                                                            |1    |
|[Wired Headphones,142223]                                                  |1    |
|[Wired Headphones,142287, Google Phone,142287, USB-C Charging Cable,142287]|1    |
|[AAA Batteries (4-pack),142452]                                            |1    |
|[USB-C Charging Cable,142577]                                              |1    |
+---------------------------------------------------------------------------+-----+
only showing top 5 rows



In [41]:
# 6. What product sold the most? 


from pyspark.sql.functions import sum

# Group the sales by product and sum the quantities
df_sales_by_product = df_sales.groupBy("Product").agg(sum("Quantity Ordered").alias("TotalQuantity"))

# Sort the products by total quantity sold in descending order
df_sales_by_product = df_sales_by_product.orderBy("TotalQuantity", ascending=False)

# Print the product with the highest total quantity sold
most_sold_product = df_sales_by_product.select("Product").first()[0]
print("The most sold product is:", most_sold_product)


The most sold product is: AAA Batteries (4-pack)
